In [1]:
from util.api_and_repo_link import link_me_up
link_me_up()
from config.environment import ENVIRONMENT
ENVIRONMENT.set_development()

C:\Users\Thomas.fuller\NotDrive\VENV\spellvenv\Scripts\python.exe
C:\Users\Thomas.fuller\NotDrive\Repos\airflow_pipelines\dags\config\environment.py
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
C:\Users\Thomas.fuller\NotDrive\Repos\airflow_pipelines\dags
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
now you just need to set_development()
C:\Users\Thomas.fuller\NotDrive\Repos\airflow_pipelines\dags\config\environment.py


# old stuff i wrote when i joind OB

In [2]:

from MAPPINGS import COUNTRY_VALUE_MAP, STATUS_VALUE_MAP, STATUS_VALUE_MAP_CSV, SUBSIDIARY_VALUE_MAP, SUBSIDIARY_VALUE_MAP_CSV, CURRENCY_VALUE_MAP, COLUMN_MAPPING, COLUMN_MAPPING_ADDRESS_BOOK, COLUMN_MAPPING_ADDRESS

import pandas
from oliverbonas_source_dags.apis.api_netsuite import APIConfig_netsuite
from util.zeep_to_df import complete_zeep_df

C:\Users\Thomas.fuller\NotDrive\VENV\spellvenv\lib\site-packages\airflow\models\base.py:49 MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)

In [3]:
from multithread_uploader import upload_all_these_records
api = APIConfig_netsuite()
from util.get_ns_number import get_ns_number

In [4]:
Customer = api.client.get_type("ns13:Customer")
CustomerSearchBasic = api.client.get_type("ns5:CustomerSearchBasic")
RecordRef = api.client.get_type("ns0:RecordRef")
CustomerAddressbookList = api.client.get_type("ns13:CustomerAddressbookList")
CustomerAddressbook = api.client.get_type("ns13:CustomerAddressbook")
Address = api.client.get_type("ns5:Address")
get_ns_number(api, "Address")

failed for ns 0
failed for ns 1
failed for ns 2
failed for ns 3
failed for ns 4
Address = api.client.get_type("ns5:Address")


In [6]:
def do_value_mappings(old_column_name, customer_dict):
    value = customer_dict[old_column_name]
    # this werid character \xa0 is removed if found...!
    if type(value) == str:
        value = value.replace("\xa0", " ")

    if old_column_name.endswith("_country"):
        return COUNTRY_VALUE_MAP[value]
    elif old_column_name == "currency":
        return CURRENCY_VALUE_MAP[value]
    elif old_column_name == "status":
        return STATUS_VALUE_MAP[STATUS_VALUE_MAP_CSV[value]]
    elif old_column_name == "subsidiary":
        return SUBSIDIARY_VALUE_MAP[SUBSIDIARY_VALUE_MAP_CSV[value]]
    else:
        return value

def to_dict(customer_series):
    customer_series[customer_series.isna()] = None # might work!
    customer_dict = customer_series.to_dict()

    # deal with value mappings here straight away
    for oldCol in customer_dict.keys():
        customer_dict[oldCol] = do_value_mappings(oldCol, customer_dict)

    # now deal with the column mappings
    for oldCol, newCol in COLUMN_MAPPING.items():
        customer_dict[newCol] = customer_dict[oldCol]
        del customer_dict[oldCol]

    addressbookList = {
        "addressbook": [
            {
                "addressbookAddress": dict(),
            },
            {
                "addressbookAddress": dict(),
            },
        ]
    }

    Addbook1 = addressbookList["addressbook"][0]
    Addbook2 = addressbookList["addressbook"][1]
    Address1 = addressbookList["addressbook"][0]["addressbookAddress"]
    Address2 = addressbookList["addressbook"][1]["addressbookAddress"]

    for oldCol, newCol in COLUMN_MAPPING_ADDRESS_BOOK.items():
        Addbook = Addbook1 if oldCol.startswith("Address1_") else Addbook2

        Addbook[newCol] = customer_dict[oldCol]
        del customer_dict[oldCol]

    for oldCol, newCol in COLUMN_MAPPING_ADDRESS.items():
        Address = Address1 if oldCol.startswith("Address1_") else Address2

        Address[newCol] = customer_dict[oldCol]
        del customer_dict[oldCol]

    customer_dict["addressbookList"] = addressbookList
    
    customer_dict["entityId"] = str(customer_dict["entityId"])
    customer_dict["externalId"] = str(customer_dict["externalId"])

    return customer_dict

In [ ]:
response = api.service_proxy.search(
    CustomerSearchBasic(),
    _soapheaders = api.build_soap_headers()
)
custs = response.body.searchResult.recordList.record

# GDPR so do a lil test

In [ ]:
test_df = pandas.read_csv("test_input_dummy_customers.csv")
test_df

In [ ]:
test_list = [Customer(**to_dict(row)) for index, row in test_df.iterrows()]

test_list[0].entityId = "customer !\"£$%^&&*()"
test_list[0].externalId = "customer !\"£$%^&&*()"

test_list[1].entityId = "﷽" * 83
test_list[1].externalId = "wideboy"

test_list[2].entityId = "ꙍ 𝁢 𝋤 ↳ ⁌ 〼 ✛ ⦛ ⺣ 🜹 ꡞ ☰ 𒋾 🯂 ⵙ 𐬑 ᚄ ʮ ⠿ "
test_list[2].externalId = "unicode"

test_list[3].entityId = "😂😂😂😂😂😂😂😂😂😂"
test_list[3].externalId = "lol"


complete_zeep_df(test_list)
upload_all_these_records(test_list, api, "addList")

In [ ]:
customer = Customer(
    entityId = "thirteen alex made this",
    externalId = "thirteen alex made this",
    firstName = "Pixie",
    lastName = "Savage",
    subsidiary = RecordRef(internalId = "1"),
    addressbookList = CustomerAddressbookList(
        addressbook = [
             
           
        ]
    )
)

upload_all_these_records([customer], api, "updateList")

# tom-f-data-team-test-

In [ ]:
PREFIX = "tom_f__DATA_TEAM__customer_for_transaction__"
test_customers = []

for index, row in pandas.concat([test_df, test_df, test_df, test_df, test_df]).reset_index(drop=True).iterrows():
    cust = PREFIX + "0" * (3 - len(str(index + 40))) + str(index+40) 
    
    row["entityId"] = cust
    row["externalId"] = cust
    test_customers.append(Customer(**to_dict(row)))

In [ ]:
upload_all_these_records(test_customers, api, "addList")

# let's do alex customers

In [5]:
# let's reload first

In [6]:
df = pandas.read_csv("ben.csv")
df

,externalId,entityId,isPerson,companyName,firstName,lastName,subsidiary,email,status,Currency,...,Address2_AddressName,Address2_attention,Address2_phone,Address2_line1,Address2_city,Address2_state,Address2_zipCode,Address2_country,Address2_defaultBillTo,Address2_defaultShipTo
0,456625,456625,True,NaN,Ben,Curtis,Parent Company,ben.curtis@oliverbonas.com,CustomER-Closed Won,GBP,...,Ship-To,Ben Curtis,7968367319,Unit F\nDavis Road Industrial Park,Chessington,Surrey,KT9 1TQ,United Kingdom,False,True
1,456625,456625,True,NaN,Ben,Curtis,Parent Company,ben.curtis@oliverbonas.com,CustomER-Closed Won,GBP,...,Ship-To,Ben Curtis,894208204,Derry Park\nTOORMAKEADY,CLAREMORRIS,COUNTY MAYO,F12 PR50,Ireland,False,False
2,456626,456626,True,NaN,Ben2,Curtis2,Parent Company,ben2.curtis2@oliverbonas.com,CustomER-Closed Won,GBP,...,Ship-To,Ben Curtis,7968367319,Unit F\nDavis Road Industrial Park,Chessington,Surrey,KT9 1TQ,United Kingdom,False,True
3,456626,456626,True,NaN,Ben2,Curtis2,Parent Company,ben2.curtis2@oliverbonas.com,CustomER-Closed Won,GBP,...,Ship-To,Ben Curtis,894208204,Derry Park\nTOORMAKEADY,CLAREMORRIS,COUNTY MAYO,F12 PR50,Ireland,False,False


In [15]:
df.columns

Index(['externalId', 'entityId', 'isPerson', 'companyName', 'firstName',
       'lastName', 'subsidiary', 'email', 'status', 'Currency',
       'Address1_AddressName', 'Address1_attention', 'Address1_phone',
       'Address1_line1', 'Address1_city', 'Address1_state', 'Address1_zipCode',
       'Address1_country', 'Address1_defaultBillTo', 'Address1_defaultShipTo',
       'Address2_AddressName', 'Address2_attention', 'Address2_phone',
       'Address2_line1', 'Address2_city', 'Address2_state', 'Address2_zipCode',
       'Address2_country', 'Address2_defaultBillTo', 'Address2_defaultShipTo'],
      dtype='object')

"""
string internalId
string country
string attention
string addressee
string addrPhone
string addr1
string addr2
string addr3
string city
string state
string zip
string addrText
boolean override
"""

Index(['externalId', 'entityId', 'isPerson', 'companyName', 'firstName',
       'lastName', 'subsidiary', 'email', 'status', 'Currency',
       'Address1_AddressName', 'Address1_attention', 'Address1_phone',
       'Address1_line1', 'Address1_city', 'Address1_state', 'Address1_zipCode',
       'Address1_country', 'Address1_defaultBillTo', 'Address1_defaultShipTo',
       'Address2_AddressName', 'Address2_attention', 'Address2_phone',
       'Address2_line1', 'Address2_city', 'Address2_state', 'Address2_zipCode',
       'Address2_country', 'Address2_defaultBillTo', 'Address2_defaultShipTo'],
      dtype='object')

In [22]:
COUNTRY_VALUE_MAP

{'Afghanistan': '_afghanistan',
 'Aland Islands': '_alandIslands',
 'Albania': '_albania',
 'Algeria': '_algeria',
 'American Samoa': '_americanSamoa',
 'Andorra': '_andorra',
 'Angola': '_angola',
 'Anguilla': '_anguilla',
 'Antarctica': '_antarctica',
 'Antigua and Barbuda': '_antiguaAndBarbuda',
 'Argentina': '_argentina',
 'Armenia': '_armenia',
 'Aruba': '_aruba',
 'Australia': '_australia',
 'Austria': '_austria',
 'Azerbaijan': '_azerbaijan',
 'Bahamas': '_bahamas',
 'Bahrain': '_bahrain',
 'Bangladesh': '_bangladesh',
 'Barbados': '_barbados',
 'Belarus': '_belarus',
 'Belgium': '_belgium',
 'Belize': '_belize',
 'Benin': '_benin',
 'Bermuda': '_bermuda',
 'Bhutan': '_bhutan',
 'Bolivia': '_bolivia',
 'Bosnia and Herzegovina': '_bosniaAndHerzegovina',
 'Botswana': '_botswana',
 'Bouvet Island': '_bouvetIsland',
 'Brazil': '_brazil',
 'British Indian Ocean Territory': '_britishIndianOceanTerritory',
 'British Virgin Islands': '_virginIslandsBritish',
 'Brunei Darussalam': '_brun

In [7]:
def get_customer_addresses(customer_df):
    addresses = []
    for index, c in customer_df.iterrows():
        billing = CustomerAddressbook(
            addressbookAddress = Address(
                country = COUNTRY_VALUE_MAP[c.Address1_country],
                attention = c.Address1_attention,
                addressee = c.Address1_AddressName,
                addrPhone = c.Address1_phone,
                addr1 = c.Address1_line1,
                city = c.Address1_city,
                state = c.Address1_state,
                zip = c.Address1_zipCode,
            ),
            defaultBilling = c.Address1_defaultBillTo,
        )
        shipping = CustomerAddressbook(
            addressbookAddress = Address(
                country = COUNTRY_VALUE_MAP[c.Address2_country],
                attention = c.Address2_attention,
                addressee = c.Address2_AddressName,
                addrPhone = c.Address2_phone,
                addr1 = c.Address2_line1,
                city = c.Address2_city,
                state = c.Address2_state,
                zip = c.Address2_zipCode,
            ),
            defaultShipping = c.Address2_defaultShipTo,
        )
        addresses.append(billing)
        addresses.append(shipping)
    
    return addresses

In [8]:
customers = []

for email, customer_df in df.groupby("email"):
    c = customer_df.iloc[0]
    
    customer = Customer(
        entityId = c.entityId,
        externalId = c.externalId,
        isPerson = c.isPerson,
        companyName = c.companyName,
        firstName = c.firstName,
        lastName = c.lastName,
        subsidiary = RecordRef(internalId=SUBSIDIARY_VALUE_MAP[c.subsidiary]["internalId"]),
        email = c.email,
        entityStatus = RecordRef(internalId = STATUS_VALUE_MAP[c.status]["internalId"]),
        currency = RecordRef(internalId = CURRENCY_VALUE_MAP[c.Currency]["internalId"]),
        addressbookList = CustomerAddressbookList(
            addressbook = get_customer_addresses(customer_df)
        )
    )
    
    customers.append(customer)

In [9]:

upload_all_these_records(customers, api, "updateList")

> [1] records in this chunk
> [1] records in this chunk

THREADS NEEDED: 2 

thread (0) spawned
thread (1) spawned
+++++++++ new thread (1) +++++++++
+++++++++ new thread (0) +++++++++
# thread (1) total [1] records [1]  success [1] failure [0]
0:00:08.260607
+++++++++ thread completed (1) +++++++++
# thread (0) total [2] records [1]  success [1] failure [0]
0:00:08.922773
+++++++++ thread completed (0) +++++++++
